<a href="https://colab.research.google.com/github/r-meleshko/deep-learning-with-python/blob/main/Chapter_12_Best_practices_for_the_real_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter optimization

In [ ]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


# A KerasTuner model-building function
def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units=units, activation="relu"),
        layers.Dense(units=10, activation="softmax"),
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    return model


# A KerasTuner HyperModel
import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build_model(hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units=units, activation="relu"),
            layers.Dense(units=num_classes, activation="softmax"),
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

        return model

In [ ]:
!mkdir -p mnist_kt_test

# Define a Bayesian Optimization tuner
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=100,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True
)

tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [ ]:
# Search for best hyperparameters
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000

x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]

tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

In [ ]:
# Retrieve best hyperparameters
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)
# Loop through the best hyperparameters and display them
for i, hps in enumerate(best_hps):
    print(f"Model {i + 1}:")
    for param_name in hps.values.keys():
        print(f"{param_name}: {hps.get(param_name)}")
    print("------------------------")

Model 1:
units: 64
optimizer: adam
------------------------
Model 2:
units: 64
optimizer: adam
------------------------
Model 3:
units: 64
optimizer: adam
------------------------
Model 4:
units: 64
optimizer: adam
------------------------


In [ ]:
# Use the validation set to find the best number of epochs for a given model
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

# For each of the best 4 models, evaluate the best number of epochs and then train the model on the full training set (train + val)
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps[:1]:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

In [ ]:
best_models[0].evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0881 - accuracy: 0.9755


[0.08805830776691437, 0.9754999876022339]

In [ ]:
# Option B: Load the best models from tuner without retraining them on the full training set with an optimal number of epochs
best_models = tuner.get_best_models(top_n)


KerasTuner attempts to provide premade search spaces that are relevant to broad categories of problems, such as image classification. Just add data, run the search, and get a pretty good model. You can try the hypermodels kt.appli- cations.HyperXception and kt.applications.HyperResNet, which are effectively tunable versions of Keras Applications models.

In [ ]:
# Automated machine learning
https://github.com/keras-team/autokeras

# Premade search spaces
# Examples: kt.applications.HyperXception and kt.applications.HyperResNet